In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from string import punctuation
from random import sample
from sklearn.feature_extraction.text import CountVectorizer


In [2]:

df1 = pd.read_csv("Data/Eclipse/EP_dup.csv", sep=";")
df2 = pd.read_csv("Data/Eclipse/EP_nondup.csv", sep=";")
df3 = pd.read_csv("Data/Mozilla/M_Duplicate BRs.csv", sep=";")  
df4 = pd.read_csv("Data/Mozilla/M_NonDuplicate BRs.csv", sep=";")
df5 = pd.read_csv("Data/ThunderBird/dup_TB.csv", sep=";")
df6 = pd.read_csv("Data/ThunderBird/Nondup_TB.csv", sep=";")

frames=[df1,df2,df3,df4,df5,df6]
data=pd.concat(frames)
data

,Issue_id,Duplicated_issue,Title1,Description1,Title2,Description2,Label
0,25,28126,cvs ui need vcm prefs default repo connection gc,it would be helpful if there was a notion of d...,wizards patch standard public cvs repositories,this patch adds a convenient way to check thin...,1
1,40,20,need connect to team stream gcqpkw,i would like to be able to connect to a team s...,workspace files,thought it would be useful if the set of repo ...,1
2,48,22,make sure can future store other project refer...,project references come in three flavours . p...,persist sharing recommendations and project ve...,project descriptions dont store sharing recomm...,1
3,61,60,.vcmmeta showing as change gdqtgw,useruser install drop into declipse user ...,need custom .vcmignore comparemerge gdqt,useruser install drop into declipse user ...,1
4,94,2,repositories view all file types open to the t...,when browsing files in the repositories view i...,opening repository resources doesnt honor type...,opening repository resource open the default ...,1
...,...,...,...,...,...,...,...
9900,418747,953052,thunderbird outlook migrate lost outlooks pst ...,i installed thunderbird hoping i could impor...,i am using thunderbird esr latest but i keep ...,i am using thunderbird .. esr latest but i...,0
9901,418821,953165,urls in mail with comma opens with c in browser,when clicking in text mail not html on url w...,picture in feed not showing,. i linked the rssfeed from httpwww.nu.nlf...,0
9902,418832,953249,greek attachments not displayed correctly,i am using gmail imap accounts. when i recei...,add the flagcolpng to linux,httpmxr.mozilla.orgcommcentralsearchstringflag...,0
9903,419035,953375,branch only random crash when i want to displ...,the problem appears with icedove the debian ...,replace undefined gpromptservice with services...,patch v discovered a call to undefined gprom...,0


In [3]:
data = data.reset_index()

In [4]:
data['TitleDescription1'] = data['Title1'].str.cat(data['Description1'],sep=" ")
data['TitleDescription2'] = data['Title2'].str.cat(data['Description2'],sep=" ")

In [5]:
stop_words = set(stopwords.words('english'))

In [6]:
def words(text, remove_stop_words=True, stem_words=False):
    # Remove punctuation from questions
    text = ''.join([c for c in text if c not in punctuation])
    
    # Lowering the words in questions
    text = text.lower()
    
    # Remove stop words from questions
    if remove_stop_words:
        text = text.split()
        text = [w for w in text if not w in stop_words]
        text = " ".join(text)
    
    # Return a list of words
    return(text)

In [7]:
def process(bug_list, bugs):
    for bug in bugs:
        bug_list.append(words(bug))
processed_bug1 = []
processed_bug2 = []
process(processed_bug1, data.TitleDescription1)
process(processed_bug2, data.TitleDescription2)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(analyzer = 'word',
                        stop_words = 'english',
                        lowercase = True,
                        max_features = 300,
                        norm = 'l1')

In [25]:
cv = CountVectorizer(max_features = 231382)
words = pd.concat([data.TitleDescription1, data.TitleDescription2], axis = 0)

cv.fit(data.TitleDescription1)
cv.fit(data.TitleDescription2)
duplicate_1=cv.transform(data.TitleDescription1)
duplicate_2=cv.transform(data.TitleDescription2)


In [18]:
tfidf.fit(words)
duplicate_1 = cv.transform(data.TitleDescription1)
duplicate_2 = cv.transform(data.TitleDescription2)

NameError: name 'tfidf' is not defined

In [26]:
x = abs(duplicate_1 - duplicate_2)
y = data['Label']

In [27]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Conv1D, GlobalMaxPooling1D


model = Sequential()
model.add(Dense(64,input_shape=(231382,)))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 64)                14808512  
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
activation_9 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
activation_10 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                

In [29]:
results = model.fit(x_train.todense(),y_train,batch_size=64,epochs=5,verbose=1,validation_data=(x_test.todense(),y_test))

MemoryError: Unable to allocate 168. GiB for an array with shape (97660, 231382) and data type int64

In [ ]:
from sklearn.metrics import classification_report
pred = model.predict_classes(x_test)
print(classification_report(y_test,pred,labels=[1,0]))